In [4]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import winsound
import datetime as dt
import os

import warnings
warnings.filterwarnings('ignore')

In [2]:
def add_diff_list(df):
    new_list = df['Close']
    diff_list = []
    for i in range(0,len(new_list)-1):
        diff = new_list[i+1] - new_list[i]
        diff_list.append(diff)
    df = df[1:len(df)]
    df['diff'] = diff_list
    df = df.reset_index(drop=True)
    return(df)    


def new_df(roll_len):
    col_list = []

    for i in range(roll_len):
        col_list.append(i)
    
    col_list.append('TS')
    return(col_list)


def reorder_df(df, roll_len, col_name):
    tot_list = []
    for i, val in tqdm(df.iterrows()):
        act_row = i+roll_len-1
        l = list(df[col_name][i:act_row+1])    
        if len(l) == roll_len:
            tot_list.append(l)
            l.append(df['TS'][act_row])
    df1=pd.DataFrame(tot_list,columns=col_list)
    return(df1)


def get_year(row):
    year_val = dt.datetime.strptime(row['TS'], '%Y-%m-%d %H:%M').date().year
    return(year_val)

def get_month(row):
    month_val = dt.datetime.strptime(row['TS'], '%Y-%m-%d %H:%M').date().month
    return(month_val)

def get_day(row):
    day_val = dt.datetime.strptime(row['TS'], '%Y-%m-%d %H:%M').date().day
    return(day_val)   

def get_hour(row):
    hour_val = dt.datetime.strptime(row['TS'], '%Y-%m-%d %H:%M').time().hour
    return(hour_val)   

def get_min(row):
    min_val = dt.datetime.strptime(row['TS'], '%Y-%m-%d %H:%M').time().minute
    return(min_val)   


def get_dir(row):
    global diff_pip
    #diff = row[roll_len-1] - row[roll_len-2]
    diff = row[roll_len-1]
    
    if diff > diff_pip:
        direction = 'increase'
    elif diff < -diff_pip:
        direction = 'decrease'
    else:
        direction = 'same'        
    
    return(direction)


def split_ts(df1, diff_pip):
    #df1['year'] = df1.apply(get_year, axis=1)
    df1['month'] = df1.apply(get_month, axis=1)
    df1['day'] = df1.apply(get_day, axis=1)
    df1['hour'] = df1.apply(get_hour, axis=1)
    df1['min'] = df1.apply(get_min, axis=1)
    df1['direction'] = df1.apply(get_dir, axis=1)
    
    cols = list(df1)

    cols.insert(0, cols.pop(cols.index('min')))
    cols.insert(0, cols.pop(cols.index('hour')))
    cols.insert(0, cols.pop(cols.index('day')))
    cols.insert(0, cols.pop(cols.index('month')))
    #cols.insert(0, cols.pop(cols.index('year')))

    df1 = df1[cols]

    del df1['TS']
    return(df1)

In [5]:
year = 2019

data = {}
data['number_of_ticks'] = 600
data['rsi_window'] = 14
data['sma_len'] = 5
data['lma_len'] = 10

diff_col = 'sema'
#diff_col = 'tick_avg'

data['pip_diff'] = 0.0001

source_file_path = f'data/yearly_tick_data/{year}.csv'
path, file_name = os.path.split(source_file_path)

target_file_name = 'tab_'+file_name
target_file_path = os.path.join(path, target_file_name)

chunk_file_name = 'chunk_'+file_name
chunk_file_path = os.path.join(path, chunk_file_name)

print(f'source_file_path : {source_file_path}')
print(f'chunk_file_path : {chunk_file_path}')
print(f'target_file_path : {target_file_path}')

source_file_path : data/yearly_tick_data/2019.csv
chunk_file_path : data/yearly_tick_data\chunk_2019.csv
target_file_path : data/yearly_tick_data\tab_2019.csv


In [6]:
%%time
df = pd.read_csv(source_file_path, nrows=100000)
#df = pd.read_csv(source_file_path)
df.head()

Wall time: 71.8 ms


,DateTime,Bid,Ask,Volume
0,20190101 22:02:37.254,1.14598,1.14682,4
1,20190101 22:02:38.590,1.14599,1.14682,2
2,20190101 22:02:39.138,1.14599,1.14684,4
3,20190101 22:02:55.787,1.14598,1.14684,4
4,20190101 22:03:02.060,1.14598,1.14684,4


In [3]:
raw_df = pd.read_csv('data\M1_2019.csv')
df_tot = raw_df[['TS', 'Close']]
df_tot.head()

,TS,Close
0,2019-01-01 17:02,1.14598
1,2019-01-01 17:03,1.14607
2,2019-01-01 17:04,1.14606
3,2019-01-01 17:05,1.14621
4,2019-01-01 17:06,1.14665


In [4]:
roll_len = 15
diff_pip = 0.0001
min_bar = 15

In [5]:
#df = df[:10000]
df = df_tot.iloc[::min_bar]
print(f'DF len : {len(df)}')
print(df.head())
df = df.reset_index(drop=True)

DF len : 24836
                  TS    Close
0   2019-01-01 17:02  1.14598
15  2019-01-01 17:17  1.14607
30  2019-01-01 17:32  1.14607
45  2019-01-01 17:47  1.14628
60  2019-01-01 18:02  1.14655


In [6]:
col_list = new_df(roll_len)
diff_df = add_diff_list(df)
rolled_df = reorder_df(diff_df, roll_len, 'diff')

24835it [00:04, 5613.41it/s]


In [7]:
cols = list(rolled_df)
cols.insert(0, cols.pop(cols.index('TS')))
rolled_df = rolled_df[cols]

In [8]:
rolled_df = split_ts(rolled_df, diff_pip)

In [9]:
print(rolled_df['direction'].value_counts())
print('--------------------------')
print(rolled_df['direction'].value_counts(normalize=True))

same        9012
increase    7966
decrease    7843
Name: direction, dtype: int64
--------------------------
same        0.363080
increase    0.320938
decrease    0.315982
Name: direction, dtype: float64


In [10]:
del rolled_df[roll_len-1]
rolled_df.to_csv('data/rolled_df.csv', index = False)
winsound.PlaySound('C:\\Windows\\Media\\tada.wav', winsound.SND_ASYNC)

In [11]:
rolled_df.head(10)

,month,day,hour,min,0,1,2,3,4,5,6,7,8,9,10,11,12,13,direction
0,1,1,20,47,0.00009,0.00000,0.00021,0.00027,0.00002,0.00005,-0.00026,-0.00050,0.00034,-0.00056,-0.00004,-0.00009,-0.00019,-0.00011,decrease
1,1,1,21,2,0.00000,0.00021,0.00027,0.00002,0.00005,-0.00026,-0.00050,0.00034,-0.00056,-0.00004,-0.00009,-0.00019,-0.00011,-0.00023,increase
2,1,1,21,17,0.00021,0.00027,0.00002,0.00005,-0.00026,-0.00050,0.00034,-0.00056,-0.00004,-0.00009,-0.00019,-0.00011,-0.00023,0.00029,same
3,1,1,21,32,0.00027,0.00002,0.00005,-0.00026,-0.00050,0.00034,-0.00056,-0.00004,-0.00009,-0.00019,-0.00011,-0.00023,0.00029,-0.00001,decrease
4,1,1,21,47,0.00002,0.00005,-0.00026,-0.00050,0.00034,-0.00056,-0.00004,-0.00009,-0.00019,-0.00011,-0.00023,0.00029,-0.00001,-0.00034,decrease
5,1,1,22,2,0.00005,-0.00026,-0.00050,0.00034,-0.00056,-0.00004,-0.00009,-0.00019,-0.00011,-0.00023,0.00029,-0.00001,-0.00034,-0.00059,increase
6,1,1,22,17,-0.00026,-0.00050,0.00034,-0.00056,-0.00004,-0.00009,-0.00019,-0.00011,-0.00023,0.00029,-0.00001,-0.00034,-0.00059,0.00025,same
7,1,1,22,32,-0.00050,0.00034,-0.00056,-0.00004,-0.00009,-0.00019,-0.00011,-0.00023,0.00029,-0.00001,-0.00034,-0.00059,0.00025,0.00006,increase
8,1,1,22,47,0.00034,-0.00056,-0.00004,-0.00009,-0.00019,-0.00011,-0.00023,0.00029,-0.00001,-0.00034,-0.00059,0.00025,0.00006,0.00016,decrease
9,1,1,23,2,-0.00056,-0.00004,-0.00009,-0.00019,-0.00011,-0.00023,0.00029,-0.00001,-0.00034,-0.00059,0.00025,0.00006,0.00016,-0.00021,decrease
